In [2]:
import pyaudio
import numpy as np

# Transmitter

## Char-Freq Table

We should compute this as fast as possible and then start transmitting to the receiver. **Figure out if there are some frequencies that we should let the more common characters take.** If so, we can create a Huffman Encoding table but that's going to involve a lot of overhead so make sure it's worth it. Start transmitting as soon as this is done building and transmit asynchronously while doing following step. 

## Encode Chars to Frequencies

Separate .txt file into chunks. Give each char in a chunk a phase and then encode frequencies according to char-freq table. **There doesn't have to be a one-to-one correspondence between sinusoid and char.** If I can fit multiple chars into one sinusoid, that's better. Only ~50 commonly used characters and thousands of frequencies. 

## Transmit 

We have to transmit according to some ECC that allows for ambient noise and sudden noise. 

# Receiver

## Start Recording Audio

## Receive Char-Freq Table

## Decoding 

Listen for chunk, receive it, then decode and write to file **asynchronously** while listening to next chunk. 

### Listen for Chunk

### Decode Chunk

Use ECC. The goal is to do a one-pass over the file so we need to have *lots* of redundancy and capability to recover from errors. 

### Write to File